**Docker Command:** 

```
docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=SQLServer123@" -p 1433:1433 --name sqlserver -d mcr.microsoft.com/mssql/server:2022-latest 
```

**Connection String:** 

```
Server = localhost,1433; Database = RMS; User Id = sa; Password = SQLServer123@; TrustServerCertificate=True
```

Bảng **Role**

In [ ]:
CREATE TABLE Roles
(
    RoleID INT PRIMARY KEY ,
    RoleName NVARCHAR(50) NOT NULL
)

In [ ]:
INSERT INTO Roles(RoleID, RoleName) VALUES (1, 'Admin');
INSERT INTO Roles(RoleID, RoleName) VALUES (2, 'Employee');
INSERT INTO Roles(RoleID, RoleName) VALUES (3, 'Company');
INSERT INTO Roles(RoleID, RoleName) VALUES (4, 'Candidate');

Bảng **Accounts**

In [ ]:
CREATE TABLE Accounts (
    AccountID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
    Username NVARCHAR(255) NOT NULL,
    PasswordHash NVARCHAR(128) NOT NULL,
    PasswordSalt NVARCHAR(128) NOT NULL,
    RoleID INT,
    FOREIGN KEY (RoleID) REFERENCES Roles(RoleID)
);


Bảng **Companies**

In [ ]:
CREATE TABLE Companies (
    CompanyID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
    AccountID UNIQUEIDENTIFIER,
    CompanyName NVARCHAR(255) NOT NULL,
    TaxIdentificationNumber NVARCHAR(255) NOT NULL,
    Representative NVARCHAR(255) NOT NULL,
    Address NVARCHAR(255),
    Email NVARCHAR(255),
    FOREIGN KEY (AccountID) REFERENCES Accounts(AccountID),
    UNIQUE (CompanyName, TaxIdentificationNumber)
);


In [ ]:
CREATE PROCEDURE RegisterCompany
    @Username NVARCHAR(50),
    @PasswordHash NVARCHAR(128),
    @PasswordSalt NVARCHAR(128),
    @CompanyName NVARCHAR(100),
    @TaxIdentificationNumber NVARCHAR(50),
    @Representative NVARCHAR(100),
    @Address NVARCHAR(200),
    @Email NVARCHAR(100)
AS
BEGIN
    IF NOT EXISTS (SELECT 1 FROM Companies WHERE CompanyName = @CompanyName)
    BEGIN
        INSERT INTO Accounts(Username, PasswordHash, PasswordSalt, RoleID)
        VALUES (@Username, @PasswordHash, @PasswordSalt, 3);
        INSERT INTO Companies(AccountID, CompanyName, TaxIdentificationNumber, Representative, Address, Email)
        VALUES (SCOPE_IDENTITY(), @CompanyName, @TaxIdentificationNumber, @Representative, @Address, @Email);
    END
END
CREATE PROCEDURE LoginUser
    @Username NVARCHAR(50),
    @PasswordHash NVARCHAR(128)
AS
BEGIN
    SELECT AccountID, RoleID FROM Accounts
    WHERE Username = @Username AND PasswordHash = @PasswordHash
END